In [35]:
import pymongo
import dns
import pandas as pd
import io
import datetime

In [36]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic
INTER_TRAVEL_TIME_AVG=5 #average travelling time from gantryFrom to gantryTo in minute
INTERCHAGE_TRAVELTO_GANTRY_TIME_AVG=3  #average travelling time from interchange to 1st gantryTo in minute

In [37]:
def thruGantry(gantryID, begin, timeRange):
    #begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['M03A']
    match={"GantryId":gantryID, "TimeInterval": {"$gte": begin, "$lt": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}   #將各車型分開加總
    result = coll.aggregate([
            { "$match": match },
            { "$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [38]:
def offInterchange(gantryID, begin, timeRange):
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['M08A']
    match={"GantryTo":gantryID, "TimeInterval": {"$gte": begin, "$lt": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}
    result = coll.aggregate([
            {"$match": match},
            {"$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [39]:

def intoInterchange(gantryFrom, gantryTo , timeStamp, timeRange):
    
    fromGntry=thruGantry(gantryFrom, timeStamp-datetime.timedelta(minutes =INTER_TRAVEL_TIME_AVG), timeRange)
    fromGntry.total = fromGntry.total.apply(lambda x:-x) # to transfer add  to minus
    #print('fromGntry',fromGntry)
    toGntry=thruGantry(gantryTo , timeStamp, timeRange)
    #print('toGntry', toGntry)
    diff=toGntry.set_index('_id').add(fromGntry.set_index('_id'), fill_value=0).reset_index()
    #print('diff',diff)
    
    offIntr=offInterchange(gantryFrom,timeStamp,timeRange)
    #print('offIntr',offIntr)
    result=diff.set_index('_id').add(offIntr.set_index('_id'), fill_value=0).reset_index()
    result.sort_values('_id', inplace=True)
    result.rename(columns={'_id':'VehicleType'}, inplace=True)
    result.total = result.total.astype(int) 
    print(result)
    return result

In [50]:
def formatDF(result):
    result.sort_values('_id', inplace=True)
    result.rename(columns={'_id':'VehicleType'}, inplace=True)
    result.total = result.total.astype(int) 
    print(result)
    return result

In [51]:

def compareIntoInterChangeDiff(gantryFrom, gantryTo, timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") + datetime.timedelta(minutes =INTERCHAGE_TRAVELTO_GANTRY_TIME_AVG) #"%Y-%m-%d %H:%M")
    print('\n', 'After time: ',timeStamp, '-', timeRange, 'min. duration')
    dfA=intoInterchange(gantryFrom, gantryTo, begin, timeRange)
    print('Before time: ',timeStamp, '-', timeRange, 'min. duration')
    delta=datetime.timedelta(minutes = timeRange)
    dfB=intoInterchange(gantryFrom, gantryTo, begin-delta, timeRange)
    print('\n---------------')

In [52]:

def compareOffInterChangeDiff(gantryTo, cmsToGantryTimeDelta, timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") + datetime.timedelta(minutes =cmsToGantryTimeDelta)
    print('\n', 'After time:',timeStamp, '-', timeRange, 'min. duration')
    dfA=offInterchange(gantryTo, begin, timeRange)
    formatDF(dfA)
    #print(dfA)
    print('Before time: ',timeStamp, '-', timeRange, 'min. duration')
    
    delta=datetime.timedelta(minutes = timeRange)
    dfB=offInterchange(gantryTo, begin-delta, timeRange)
    formatDF(dfB)
    #print(dfB)
    print('\n')

In [53]:
print('The CMS infoed: 204~198K壅塞車速40以下請改道','\n')
compareIntoInterChangeDiff('01F2156N','01F2089N',"2019/04/05 13:21",60) #2019-04-05 13:21

print('\n','The time travelling compared: ','\n')
compareIntoInterChangeDiff('01F2156N','01F2089N',"2019/04/04 13:21",60)
compareIntoInterChangeDiff('01F2156N','01F2089N',"2019/04/06 13:23",60)
compareIntoInterChangeDiff('01F2156N','01F2089N',"2019/04/07 13:21",60)


The CMS infoed: 204~198K壅塞車速40以下請改道 


 After time:  2019/04/05 13:21 - 60 min. duration
   VehicleType  total
0            5     -1
1           31    444
2           32     83
3           41      5
4           42      8
Before time:  2019/04/05 13:21 - 60 min. duration
   VehicleType  total
0            5     -2
1           31    685
2           32    202
3           41      7
4           42     18

---------------

 The time travelling compared:  


 After time:  2019/04/04 13:21 - 60 min. duration
   VehicleType  total
0            5      6
1           31    577
2           32    202
3           41      0
4           42     26
Before time:  2019/04/04 13:21 - 60 min. duration
   VehicleType  total
0            5      9
1           31    345
2           32     84
3           41      8
4           42     14

---------------

 After time:  2019/04/06 13:23 - 60 min. duration
   VehicleType  total
0            5      4
1           31    688
2           32    143
3           41     20
4 

In [54]:
print('The CMS infoed: 204~198K壅塞車速40以下請改道','\n')
compareOffInterChangeDiff('01F2156N', 4, "2019/06/25 06:55",60) #2019-04-05 13:21

print('\n','\n','---The time travel compared---','\n')
compareOffInterChangeDiff('01F2156N', 4, "2019/06/26 06:55",60) #2019-04-05 13:21
compareOffInterChangeDiff('01F2156N', 4, "2019/06/27 06:55",60) #2019-04-05 13:21
compareOffInterChangeDiff('01F2156N', 4, "2019/06/24 06:55",60) #2019-04-05 13:21

The CMS infoed: 204~198K壅塞車速40以下請改道 


 After time: 2019/06/25 06:55 - 60 min. duration
   VehicleType  total
2            5     10
1           31    188
0           32     73
3           42     27
Before time:  2019/06/25 06:55 - 60 min. duration
   VehicleType  total
2            5     10
1           31     81
0           32     43
3           41      1
4           42     23



 
 ---The time travel compared--- 


 After time: 2019/06/26 06:55 - 60 min. duration
   VehicleType  total
0            5      3
2           31    134
3           32     37
1           41      1
4           42     13
Before time:  2019/06/26 06:55 - 60 min. duration
   VehicleType  total
3            5      1
0           31     40
4           32     22
1           41      2
2           42     16



 After time: 2019/06/27 06:55 - 60 min. duration
   VehicleType  total
2            5      2
3           31    140
0           32     25
1           42     13
Before time:  2019/06/27 06:55 - 60 min. duration
   Ve